In [1]:
%matplotlib inline

import dask.array as da
from multipledispatch import dispatch
import matplotlib.pyplot as plt
import numpy as np

In [2]:
@dispatch(np.ndarray,np.ndarray)
def dot(A,B):
    return np.dot(A,B)

@dispatch(da.Array,da.Array)
def dot(A,B):
    return da.dot(A,B)

@dispatch(np.ndarray)
def diag(A):
    return np.diag(np.diag(A))

@dispatch(da.Array)
def diag(A):
    return da.diag(da.diag(A))

@dispatch(np.ndarray)
def inv_diag(A):
    return np.diag(1/np.diag(A))

@dispatch(da.Array)
def inv_diag(A):
    return da.diag(1/da.diag(A))

@dispatch(np.ndarray)
def absolute(A):
    return np.absolute(A)

@dispatch(da.Array)
def absolute(A):
    return da.absolute(A)

In [3]:
n = 5000
seed = 20009
np.random.seed(seed)

A = np.random.normal(0,1,size=(n,n))
A = A + A.transpose() # symmetrize
np.fill_diagonal(A,n) # diagonally dominant
x = np.random.normal(10,3,size=(n,1))
b = A.dot(x)

A_da = da.from_array(A, chunks=(1000,1000))
x_da = da.from_array(x, chunks=(1000,1))
b_da = da.from_array(b, chunks=(1000,1))

## Solving Linear Systems

### Using the API

In [4]:
%%time
x0 = np.linalg.solve(A,b)

CPU times: user 4.67 s, sys: 150 ms, total: 4.82 s
Wall time: 1.72 s


In [5]:
np_err = np.absolute(x0-x)**2
np_err.sum()

8.7962369726464375e-25

In [6]:
%%time
x0 = da.linalg.solve(A_da,b_da)
x0 = x0.compute()

CPU times: user 7.61 s, sys: 437 ms, total: 8.05 s
Wall time: 2.51 s


In [7]:
da_err = da.absolute(x0-x_da)**2
da_err.sum().compute()

8.5249601565520921e-25

### Using Jacobi

In [8]:
def jacobi(A,b,init):
    
    ## set up parameters
    tol = 1e-8
    omega = 0.75
    MAX_ITER = 100
    count = 0
    
    ## set up relevant matrices
    D = diag(A)
    Dinv = inv_diag(A)
    main = dot(Dinv,A-D)
    xplus = init
    const = dot(Dinv,b)
    
    converged = False
    
    while not converged:
        
        count += 1
        old = xplus
        xplus = omega*(const - dot(main,xplus)) + (1-omega)*xplus
        err = absolute(old-xplus)**2
        err = da.compute(err.sum())[0]
        
        print("{0} : {1}".format(count,err))
        
        if ((err<tol) or (count>MAX_ITER)):
            converged = True
            
    return xplus

In [9]:
init = np.ones((n,1))/n
init_da = da.from_array(init, chunks=(1000,1))

In [10]:
%%time
jacobi(A,b,init)

1 : 304816.72800748266
2 : 19072.783901650542
3 : 1210.5058295323317
4 : 77.8994704541918
5 : 5.080203974947527
6 : 0.3355154185454569
7 : 0.02242292084144895
8 : 0.001515160347123608
9 : 0.00010342843283987332
10 : 7.126349493380152e-06
11 : 4.952033272182116e-07
12 : 3.467785423956799e-08
13 : 2.4454565423544115e-09
CPU times: user 14.4 s, sys: 336 ms, total: 14.8 s
Wall time: 4.17 s


array([[  4.98054334],
       [  9.59376154],
       [  8.17369321],
       ..., 
       [ 10.1059967 ],
       [  8.06264909],
       [  8.63343822]])

In [11]:
%%time
jacobi(A_da,b_da,init_da)

1 : 304816.72800748266
2 : 19072.78390165054
3 : 1210.505829532332
4 : 77.89947045419181
5 : 5.08020397494753
6 : 0.3355154185454551
7 : 0.022422920841448674
8 : 0.001515160347123639
9 : 0.00010342843283991032
10 : 7.126349493373324e-06
11 : 4.95203327220078e-07
12 : 3.467785423953907e-08
13 : 2.4454565422468767e-09
CPU times: user 4min 14s, sys: 10.5 s, total: 4min 25s
Wall time: 1min 9s


dask.array<add-626..., shape=(5000, 1), dtype=float64, chunksize=(1000, 1)>

I must be thinking of something incorrectly with `da.compute()` and how I organize this...